In [1]:
import numpy as np
import pandas as pd
import time
from nba_api.stats.endpoints import playercareerstats, commonplayerinfo, commonallplayers, playerprofilev2


In [2]:
nba_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)  #Retrieve all active players data from API
nba_players.get_data_frames()[0]

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_SLUG,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2021,precious_achiuwa,precious_achiuwa,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
1,203500,"Adams, Steven",Steven Adams,1,2013,2021,steven_adams,steven_adams,1610612763,Memphis,Grizzlies,MEM,grizzlies,grizzlies,Y,00
2,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2021,bam_adebayo,bam_adebayo,1610612748,Miami,Heat,MIA,heat,heat,Y,00
3,1630583,"Aldama, Santi",Santi Aldama,1,2021,2021,santi_aldama,santi_aldama,1610612763,Memphis,Grizzlies,MEM,grizzlies,grizzlies,Y,11
4,200746,"Aldridge, LaMarcus",LaMarcus Aldridge,1,2006,2021,lamarcus_aldridge,lamarcus_aldridge,1610612751,Brooklyn,Nets,BKN,nets,nets,Y,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,201152,"Young, Thaddeus",Thaddeus Young,1,2007,2021,thaddeus_young,thaddeus_young,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
614,1629027,"Young, Trae",Trae Young,1,2018,2021,trae_young,trae_young,1610612737,Atlanta,Hawks,ATL,hawks,hawks,Y,00
615,1630209,"Yurtseven, Omer",Omer Yurtseven,1,2020,2021,tmp_omer_yurtseven,omer_yurtseven,1610612748,Miami,Heat,MIA,heat,heat,Y,01
616,203469,"Zeller, Cody",Cody Zeller,0,2013,2021,cody_zeller,cody_zeller,0,,,,None,,Y,00


In [3]:
teamIdMask = (nba_players.get_data_frames()[0]['TEAM_ID'] != 0)                            #Filter prerequisites & filter dataframe columns
gamesPlayedFlagMask = (nba_players.get_data_frames()[0]['GAMES_PLAYED_FLAG'] != "N")
dismissPlayer = (nba_players.get_data_frames()[0]['PERSON_ID'] != 1630597)

                
cPlayers = nba_players.get_data_frames()[0].loc[teamIdMask & gamesPlayedFlagMask & dismissPlayer ]  #Filtered by column
cPlayers = cPlayers.rename(columns={'PERSON_ID':'PLAYER_ID','DISPLAY_FIRST_LAST':'PLAYER_NAME'}) 
cPlayers = cPlayers[['PLAYER_ID','PLAYER_NAME','TEAM_NAME']]


careerv2 = playerprofilev2.PlayerProfileV2(per_mode36="PerGame",player_id=200765).career_totals_regular_season.get_data_frame()  #pull data from endpoint & filter dataframe using playerID
secondDf = careerv2[["PLAYER_ID", "GP", "MIN", "PTS", "REB", "AST", "STL", "BLK"]]


In [4]:
careerv2 = playerprofilev2.PlayerProfileV2(per_mode36="PerGame",player_id=200765).career_totals_regular_season.get_data_frame()  #pull data from endpoint & filter dataframe using playerID
careerv2[["PLAYER_ID", "GP", "MIN", "PTS", "REB", "AST", "STL", "BLK"]]   

,PLAYER_ID,GP,MIN,PTS,REB,AST,STL,BLK
0,200765,957,29.9,9.8,4.5,7.9,1.6,0.1


In [7]:
                                              #Pull data from API commonplayerinfo for personal info    
commonheightInfo = pd.DataFrame()

commonInfo = commonplayerinfo.CommonPlayerInfo(player_id = 200765 ,timeout=30)            #ask to api
commonInfo.get_data_frames()[0].loc[(commonInfo.get_data_frames()[0]['GAMES_PLAYED_FLAG'] != 'N') & (commonInfo.get_data_frames()[0]['PERSON_ID'] != 1630597)  & (commonInfo.get_data_frames()[0]['TEAM_ID'] != 0) ]
commonPInfo = commonInfo.get_data_frames()[0]

                                              #Apply lamba to convert weight to kg
commonPInfo['WEIGHT'] = commonPInfo['WEIGHT'].astype(float).apply(lambda x: x*0.453592).round(2)
                                             
                                                                                                                      #Apply lamba to convert weight to kg
                                                                        commonheightInfo[['FOOT','INCHES']] = commonPInfo['HEIGHT'].str.split('-', expand=True)  #Splitting height in two columns
                                                                        convertedFoot = commonheightInfo['FOOT'].astype(float).apply(lambda x: x*30.48)          #lambda foot>cm
                                                                        convertedInches = commonheightInfo['INCHES'].astype(float).apply(lambda x: x*2.54)       #lambda inches>cm
                                                                        converted = convertedFoot + convertedInches                                              #sum
                                                                        commonPInfo['HEIGHT'] = converted                                                        #add into df in height column 

commonFilteredInfo = commonPInfo.filter(['DISPLAY_FIRST_LAST','TEAM_NAME','POSITION','HEIGHT','WEIGHT','COUNTRY','BIRTHDATE','SEASON_EXP','DRAFT_NUMBER'], axis=1)
commonFilteredInfo



,DISPLAY_FIRST_LAST,TEAM_NAME,POSITION,HEIGHT,WEIGHT,COUNTRY,BIRTHDATE,SEASON_EXP,DRAFT_NUMBER
0,Rajon Rondo,Cavaliers,Guard,185.42,81.65,USA,1986-02-22T00:00:00,15,21


In [8]:
                    # Next Game pulling data
nextGame = playerprofilev2.PlayerProfileV2(per_mode36="PerGame",player_id=200765).next_game.get_data_frame()  #pull data from endpoint & filter dataframe
nextGame


,GAME_ID,GAME_DATE,GAME_TIME,LOCATION,PLAYER_TEAM_ID,PLAYER_TEAM_CITY,PLAYER_TEAM_NICKNAME,PLAYER_TEAM_ABBREVIATION,VS_TEAM_ID,VS_TEAM_CITY,VS_TEAM_NICKNAME,VS_TEAM_ABBREVIATION
0,0052100201,APR 15 2022,07:30 PM,H,1610612739,Cleveland,Cavaliers,CLE,1610612737,Atlanta,Hawks,ATL


In [ ]:
               #Retrieve salary information

In [ ]:
               #Concatenate dataframes

In [ ]:
prueba = []
for ele in players["id"]:
  career = playercareerstats.PlayerCareerStats(player_id = ele)
  prueba.append(career)
  print(len(prueba))
  time.sleep(0.5)
prueba

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
len(prueba)

587

In [ ]:
data_players = prueba[0].get_data_frames()[0].loc[prueba[2].get_data_frames()[0].SEASON_ID == "2021-22"]

In [ ]:
data_players = prueba[0].get_data_frames()[0].loc[prueba[2].get_data_frames()[0].SEASON_ID == "2021-22"]
for player in range(0,len(prueba)):
  data_players = pd.concat([data_players,prueba[player].get_data_frames()[0].loc[prueba[player].get_data_frames()[0].SEASON_ID == "2021-22"]], ignore_index=True)

In [ ]:
data_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PLAYER_ID          707 non-null    object 
 1   SEASON_ID          707 non-null    object 
 2   LEAGUE_ID          707 non-null    object 
 3   TEAM_ID            707 non-null    object 
 4   TEAM_ABBREVIATION  707 non-null    object 
 5   PLAYER_AGE         707 non-null    float64
 6   GP                 707 non-null    object 
 7   GS                 707 non-null    object 
 8   MIN                707 non-null    float64
 9   FGM                707 non-null    object 
 10  FGA                707 non-null    object 
 11  FG_PCT             707 non-null    float64
 12  FG3M               707 non-null    object 
 13  FG3A               707 non-null    object 
 14  FG3_PCT            707 non-null    float64
 15  FTM                707 non-null    object 
 16  FTA                707 non

In [ ]:
data_players.drop(data_players.loc[data_players.TEAM_ID == 0].index , axis=0, inplace = True)

In [ ]:
data_players.head(15)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630173,2021-22,00,1610612761,TOR,22.0,73,28,1725.0,265,...,0.595,146,327,473,82,37,41,84,151,664
1,203500,2021-22,00,1610612763,MEM,28.0,76,75,1999.0,210,...,0.543,349,411,760,256,65,60,115,153,528
2,1628389,2021-22,00,1610612748,MIA,24.0,56,56,1825.0,406,...,0.753,137,427,564,190,80,44,148,171,1068
3,1630583,2021-22,00,1610612763,MEM,21.0,32,0,360.0,53,...,0.625,33,54,87,21,6,10,16,36,132
4,200746,2021-22,00,1610612751,BKN,36.0,47,12,1050.0,252,...,0.873,73,185,258,42,14,47,44,78,607
5,1629638,2021-22,00,1610612740,NOP,23.0,50,19,1317.0,237,...,0.722,36,128,164,139,41,19,85,88,639
6,1629638,2021-22,00,1610612762,UTA,23.0,15,2,149.0,16,...,0.917,1,22,23,17,5,4,8,15,53
8,1628960,2021-22,00,1610612749,MIL,26.0,66,61,1805.0,255,...,0.865,32,190,222,100,46,18,43,96,733
9,1628386,2021-22,00,1610612739,CLE,23.0,56,56,1810.0,369,...,0.708,192,410,602,92,44,75,94,97,904
10,1630631,2021-22,00,1610612740,NOP,23.0,54,1,834.0,131,...,0.679,25,75,100,152,71,7,40,73,330
